In [ ]:
import pydicom
import numpy as np
from PIL import Image

In [ ]:
import os
from pathlib import Path

# Get the current file's directory
current_dir = Path().absolute()

# Set working directory to parent directory (one level up)
parent_dir = current_dir.parent
os.chdir(parent_dir)

# Print confirmation
print(f"Working directory set to: {os.getcwd()}")

In [ ]:

# 1. Read the DICOM
ds = pydicom.dcmread("İskemi_veri kümesi/Diğer (Normal ve Kronik İskemi Bulgu Kesitleri)/103618.dcm")
ds2 = pydicom.dcmread("TeknofestDCM_Anon/Diğer(Normal kesit veya kronik iskemik bulgular)/96358.dcm")

In [5]:
ds

Dataset.file_meta -------------------------------
(0002,0000) File Meta Information Group Length  UL: 158
(0002,0001) File Meta Information Version       OB: b'\x00\x01'
(0002,0003) Media Storage SOP Instance UID      UI: 1.3.12.2.1107.5.1.7.120033.30000023071409082162100000251
(0002,0010) Transfer Syntax UID                 UI: JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])
(0002,0012) Implementation Class UID            UI: 1.3.6.1.4.1.30071.8
(0002,0013) Implementation Version Name         SH: 'fo-dicom 4.0.8'
-------------------------------------------------
(0018,0050) Slice Thickness                     DS: '1.5'
(0028,0002) Samples per Pixel                   US: 1
(0028,0004) Photometric Interpretation          CS: 'MONOCHROME2'
(0028,0010) Rows                                US: 609
(0028,0011) Columns                             US: 512
(0028,0030) Pixel Spacing                       DS: [0.320734375, 0.3207339901]
(0028,0100) Bits Al

In [6]:
ds2

Dataset.file_meta -------------------------------
(0002,0000) File Meta Information Group Length  UL: 196
(0002,0001) File Meta Information Version       OB: b'\x00\x01'
(0002,0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002,0003) Media Storage SOP Instance UID      UI: 1.2.392.200036.9123.100.11.15002200303521616170150170322902562
(0002,0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002,0012) Implementation Class UID            UI: 1.3.6.1.4.1.30071.8
(0002,0013) Implementation Version Name         SH: 'fo-dicom 4.0.8'
-------------------------------------------------
(0018,0050) Slice Thickness                     DS: '5.00'
(0020,0013) Instance Number                     IS: '18'
(0020,0032) Image Position (Patient)            DS: [-150.7945, -159.7945, 85.0000]
(0020,0037) Image Orientation (Patient)         DS: [1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000]
(0028,0002) Samples per Pixel                   US: 1
(0028,0004) Photometri

In [ ]:
from pathlib import Path
from PIL import Image
import os

IMAGE_EXTENSIONS = {'.dcm'}

def read_images(root_dir):
    root = Path(root_dir)
    for path in root.rglob('*'):
        if path.suffix.lower() in IMAGE_EXTENSIONS:
            img = read_dicom(path)
            yield path, img

# Usage
# parent_dir="ReadableVersion"
for path, img in read_images('TeknofestDCM_Anon'):
    current_dir = os.path.join("ReadableVersion",str(path)[:str(path).rfind("\\")])
    if(os.path.exists(current_dir) == False):
        os.makedirs(current_dir, exist_ok=True)
        print(f"Directory created: {current_dir}")
    img = Image.fromarray(img)
    img.save(os.path.join("ReadableVersion",str(path)[:-4] + ".png"))
    print(f"{path}: size={img.size}")

Directory created: ReadableVersion\TeknofestDCM_Anon\Diğer(Normal kesit veya kronik iskemik bulgular)
TeknofestDCM_Anon\Diğer(Normal kesit veya kronik iskemik bulgular)\10058.dcm: size=(512, 512)
TeknofestDCM_Anon\Diğer(Normal kesit veya kronik iskemik bulgular)\10190.dcm: size=(512, 512)
TeknofestDCM_Anon\Diğer(Normal kesit veya kronik iskemik bulgular)\10224.dcm: size=(512, 512)
TeknofestDCM_Anon\Diğer(Normal kesit veya kronik iskemik bulgular)\10230.dcm: size=(512, 512)
TeknofestDCM_Anon\Diğer(Normal kesit veya kronik iskemik bulgular)\10460.dcm: size=(512, 512)
TeknofestDCM_Anon\Diğer(Normal kesit veya kronik iskemik bulgular)\10553.dcm: size=(512, 512)
TeknofestDCM_Anon\Diğer(Normal kesit veya kronik iskemik bulgular)\1056.dcm: size=(512, 512)
TeknofestDCM_Anon\Diğer(Normal kesit veya kronik iskemik bulgular)\10659.dcm: size=(512, 512)
TeknofestDCM_Anon\Diğer(Normal kesit veya kronik iskemik bulgular)\10942.dcm: size=(512, 512)
TeknofestDCM_Anon\Diğer(Normal kesit veya kronik iske

In [9]:
arr=ds.pixel_array

In [13]:
def read_dicom(path):

    ds = pydicom.dcmread(path)
    # 2. Convert to pixel array
    arr=ds.pixel_array
    # 3. (Optional) Apply rescale slope/intercept
    slope = getattr(ds, 'RescaleSlope', 1.0)
    intercept = getattr(ds, 'RescaleIntercept', 0.0)
    arr = arr * slope + intercept

    # 4. Windowing (convert to 0–255)
    #    If the DICOM has WindowCenter/Width tags, use them:
    if ('WindowCenter' in ds) and ('WindowWidth' in ds):
        center = ds.WindowCenter
        width = ds.WindowWidth
        # Handle multi-valued tags
        if isinstance(center, pydicom.multival.MultiValue):
            center = center[0]
            width = width[0]
        low = center - width/2
        high = center + width/2
        arr = np.clip(arr, low, high)
    arr = (arr - arr.min()) / (arr.max() - arr.min())  # normalize 0–1
    arr = (arr * 255).astype(np.uint8)                  # scale to 0–255
    return arr


In [53]:
from pathlib import Path
from PIL import Image
import os

IMAGE_EXTENSIONS = {'.dcm'}

def read_images(root_dir):
    root = Path(root_dir)
    for path in root.rglob('*'):
        if path.suffix.lower() in IMAGE_EXTENSIONS:
            img = read_dicom(path)
            yield path, img

# Usage
# parent_dir="ReadableVersion"
for path, img in read_images('TeknofestDCM_Anon'):
    current_dir = os.path.join("ReadableVersion",str(path)[:str(path).rfind("\\")])
    if(os.path.exists(current_dir) == False):
        os.makedirs(current_dir, exist_ok=True)
        print(f"Directory created: {current_dir}")
    img = Image.fromarray(img)
    img.save(os.path.join("ReadableVersion",str(path)[:-4] + ".png"))
    print(f"{path}: size={img.size}")

Directory created: ReadableVersion\TeknofestDCM_Anon\Diğer(Normal kesit veya kronik iskemik bulgular)
TeknofestDCM_Anon\Diğer(Normal kesit veya kronik iskemik bulgular)\10058.dcm: size=(512, 512)
TeknofestDCM_Anon\Diğer(Normal kesit veya kronik iskemik bulgular)\10190.dcm: size=(512, 512)
TeknofestDCM_Anon\Diğer(Normal kesit veya kronik iskemik bulgular)\10224.dcm: size=(512, 512)
TeknofestDCM_Anon\Diğer(Normal kesit veya kronik iskemik bulgular)\10230.dcm: size=(512, 512)
TeknofestDCM_Anon\Diğer(Normal kesit veya kronik iskemik bulgular)\10460.dcm: size=(512, 512)
TeknofestDCM_Anon\Diğer(Normal kesit veya kronik iskemik bulgular)\10553.dcm: size=(512, 512)
TeknofestDCM_Anon\Diğer(Normal kesit veya kronik iskemik bulgular)\1056.dcm: size=(512, 512)
TeknofestDCM_Anon\Diğer(Normal kesit veya kronik iskemik bulgular)\10659.dcm: size=(512, 512)
TeknofestDCM_Anon\Diğer(Normal kesit veya kronik iskemik bulgular)\10942.dcm: size=(512, 512)
TeknofestDCM_Anon\Diğer(Normal kesit veya kronik iske